In [54]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

import bs4
from bs4 import BeautifulSoup

import requests
import pandas as pd
import time


In [14]:
#appending data frame to existing excel file
def append_df_to_excel(df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, df], ignore_index=True)
    result.to_excel(excel_path, index=False)


In [15]:
# #reading provinces list
# read_province_list = open("jawa_bali.txt",'r')
# provinces=[]
# # # for _ in range(33): #skipping province
# # #     next(read_province_list)
# for province in read_province_list:
#     provinces.append(province.replace("\n", ", Indonesia").strip())

# # provinces[33]+=", Indonesia"
# # provinces = pd.read_excel(r"Indonesias_Hotel_List.xlsx",usecols=[2],skiprows=[0]).drop_duplicates().values.tolist()


# #provinces=[provinces[44]]
# print((provinces))

In [16]:
kota = pd.read_csv(r"Indonesian_Family_Life_Survey_4_Longitude_and_Latitude.csv", usecols=[5,6]).drop_duplicates()
len(kota)
for j in range(101,102):
    print(kota.iloc[j,1])

JAKARTA SELATAN


In [58]:
#url link
my_url='https://www.traveloka.com/en-id/'


#calling web driver from its directory
s=Service(r'C:\Users\62811\Downloads\geckodriver-v0.30.0-win64\geckodriver.exe') 


def strip(x):
    if x !=None:
        return   str(x.text.strip())
    else:
        return str('')



Current province is DKI JAKARTA, JAKARTA SELATAN 101


TimeoutException: Message: 
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


In [150]:
#url link
my_url='https://www.traveloka.com/en-id/'


#calling web driver from its directory
s=Service(r'C:\Users\62811\Downloads\geckodriver-v0.30.0-win64\geckodriver.exe') 


driver = webdriver.Firefox(service=s)
wait = WebDriverWait(driver, 30)
driver.get(my_url)
search_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[data-testid='autocomplete-field']"))).send_keys("Jakarta")

wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='autocomplete-item-name' and contains(string(), 'Jakarta')]"))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='search-submit-button']"))).click()



x=''
while True:
    try:
        time.sleep(4)
        x = x + wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "_2qd8A"))).get_attribute('innerHTML') #find_elements_by_class_name("_7192d3184")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"next-button"))).click()
        print("Clicked on  Next Page »")
    except TimeoutException:
        print("No more Next Page »")
        break
driver.quit()



import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next

In [151]:
# card=BeautifulSoup(x, 'html.parser').find_all(True,{'class':'ztzlF _2nyWi'})
# len(card)

#card sudah di simpan di C:\Users\62811\Documents\Web Scraping Project (traveloka.com)

1854

In [169]:
i=0
list = [ ]
for item in card:
    name = strip(item.find(True, {'class':'_1z5je _10ZQX tvat-hotelName'}))
    
    loc = strip(item.find(True,{'class':'_3tICV'}))
    
    rating = strip(item.find(True,{'class':'tvat-ratingScore'}))
    
    rating_label = strip(item.find(True,{'class':'_16hDy'})).split()
    
    number_of_reviews = strip(item.find(True,{'class':'_30os4'}))
    
    price = strip(item.find(True,{'class':'_22n9I tvat-primaryPrice'}))
    
    # bed = strip(item.find(True,{'class':'_2075f7b46'}))
    
    # price_for= strip(item.find(True,{'class':'_4abc4c3d5 _7ee1c7d14'}))

    # stars = item.find(True,{'class':'_bebcf8d60'})

    # city = kota.iloc[j,1]

    # province_name = kota.iloc[j,0]

    # if stars is None:
    #     stars_count = '0'
    # else:
    #     stars_count = len(stars)

    # href_tag= item.find('a', href=True)['href']

    list_in_list = [name,loc,rating,rating_label,number_of_reviews,price]#,bed,price_for,stars_count,city,href_tag]
    
    list.append(list_in_list)
    #print(list_in_list)
    i=i+1
df = pd.DataFrame(list,columns= ['name','loc','rating','rating_label','number_of_reviews','price'])
df

,name,loc,rating,rating_label,number_of_reviews,price
0,"The Langham, Jakarta","Senayan, Jakarta",8.8,"[Impressive, -, 8.8(192)]",(192),Rp 3.630.000
1,Hotel Indonesia Kempinski Jakarta,"Thamrin, Jakarta",8.9,"[Impressive, -, 8.9(4284)]",(4284),Rp 3.424.300
2,"Hotel Mulia Senayan, Jakarta","Senayan, Jakarta",8.7,"[Impressive, -, 8.7(1372)]",(1372),Rp 2.056.999
3,MAY Property Apartment Kalibata City Tower Gah...,"Pancoran, Jakarta",6.9,"[Good, -, 6.9(11)]",(11),Rp 350.000
4,Leisure Inn Arion Hotel,"Rawamangun, Jakarta",8.2,"[Convenient, -, 8.2(612)]",(612),Rp 365.060
...,...,...,...,...,...,...
1849,Single Room in Mangga Besar (CER),"Mangga Besar, Jakarta",6.6,"[Good, -, 6.6(20)]",(20),Rp 180.000
1850,2BR Grand Palace Apartment Kemayoran near to J...,"Kemayoran, Jakarta",7.7,"[Convenient, -, 7.7(6)]",(6),Rp 1.215.278
1851,Hotel Permai,"Mangga Besar, Jakarta",7.7,"[Convenient, -, 7.7(396)]",(396),Rp 225.000
1852,Harapan Solata Homestay,"Kelapa Island, Jakarta",7.5,"[Convenient, -, 7.5(12)]",(12),Rp 499.999


AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [127]:
f = open("card.html", "a")
f.write(str(card))
f.close()

In [10]:
r = requests.get("https://www.traveloka.com/en-id/")

page = BeautifulSoup(r.text,'html.parser')

driver.find_elements(by=By.CLASS_NAME, value="_7192d3184")

review = [x for x in page.find_all(True, {'class':'css-1dbjc4n r-1awozwy r-18u37iz r-1777fci r-edyy15'})]
print(review)

[<div class="css-1dbjc4n r-1awozwy r-18u37iz r-1777fci r-edyy15"><div class="css-1dbjc4n r-kdyh1x r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-14qw7yl r-1wyyakw" style="background-color:rgba(255,255,255,1.00);border-top-color:rgba(205,208,209,1.00);border-right-color:rgba(205,208,209,1.00);border-bottom-color:rgba(205,208,209,1.00);border-left-color:rgba(205,208,209,1.00);border-top-width:1px;border-right-width:1px;border-bottom-width:1px;border-left-width:1px;height:calc(100% - 2);-webkit-transition-delay:0ms;transition-delay:0ms;-webkit-transition-property:width, height, border-color, border-width, background-color;transition-property:width, height, border-color, border-width, background-color;-webkit-transition-timing-function:ease;transition-timing-function:ease;width:calc(100% - 2);will-change:width, height, border-color, border-width, background-color;-moz-transition-property:width, height, border-color, border-width, background-color"></div><div class="css-1dbjc4n r-1kb76zh">

In [8]:
len(review)

2